In [1]:
from dotenv import load_dotenv
import os
import cv2
import traceback
from utilities import print_emotions
from hume import HumeStreamClient
from hume.models.config import FaceConfig

In [ ]:


async def main():
    video_path = "test.mp4" 
    frame_skip = 30  # Process every 30 frames

    try:
        client = HumeStreamClient(os.environ["HUME_API_KEY"])
        config = FaceConfig(identify_faces=True)

        # Open the video file
        cap = cv2.VideoCapture(video_path)
        current_frame = 0

        async with client.connect([config]) as socket:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Only process every 30th frame
                if current_frame % frame_skip == 0:
                    # Convert the frame from BGR to RGB (OpenCV uses BGR by default)
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    
                    # Save the frame temporarily to send it
                    temp_image_path = f"temp_frame_{current_frame}.png"
                    cv2.imwrite(temp_image_path, frame_rgb)

                    # Send the frame file
                    result = await socket.send_file(temp_image_path)
                    emotions = result["face"]["predictions"][0]["emotions"]
                    print_emotions(emotions)
                
                current_frame += 1

            cap.release()

    except Exception as e:
        print(traceback.format_exc())

await main()
